# 02 Limpieza y Preprocesamiento de Datos

Este notebook se enfoca en la limpieza y preprocesamiento del conjunto de datos para prepararlo para el análisis y modelado.

## 1. Cargar el Conjunto de Datos

In [2]:
# Importar la librería necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Cargar el conjunto de datos
file_path = '../data/raw/bank-full.csv'
data = pd.read_csv(file_path, sep=';')
data_clean = data.copy()

# Mostrar las primeras filas del conjunto de datos
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


## 2. Manejo de Valores Faltantes

In [3]:
# Identificación de valores faltantes
missing_data = data.isnull().sum()

# Verificar si hay valores faltantes en el DataFrame
if missing_data.sum() == 0:
    print("No hay valores faltantes en el DataFrame.")
else:
    print(f"Total de valores faltantes en el DataFrame: {missing_data.sum()}")

    # Filtrar las columnas con valores faltantes
    missing_data = missing_data[missing_data > 0]
    
    # Visualización de valores faltantes
    missing_data.plot(kind='bar', figsize=(12, 6))
    plt.title('Valores Faltantes por Variable')
    plt.xlabel('Variable')
    plt.ylabel('Número de Valores Faltantes')
    plt.show()

    # Ejemplo de imputación de valores faltantes con la mediana (modificar según sea necesario)
    for column in missing_data.index:
        if data[column].dtype == 'object':
            data_clean[column].fillna(data[column].mode()[0], inplace=True)
        else:
            data_clean[column].fillna(data[column].median(), inplace=True)

No hay valores faltantes en el DataFrame.


## 3. Manejo de Valores Atípicos

In [4]:
# Función para detectar valores atípicos utilizando el método del rango intercuartílico (IQR)
def detect_outliers(data: pd.DataFrame, num_var: str) -> pd.DataFrame:
    Q1 = data[num_var].quantile(0.25)
    Q3 = data[num_var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[num_var] < lower_bound) | (data[num_var] > upper_bound)]
    return outliers

# Seleccionar las variables numéricas
num_vars = data.select_dtypes(include=['int64']).columns
print(num_vars)

# Detectar valores atípicos en todas las variables numéricas
outliers = []
for num_var in num_vars:
    outliers.append(detect_outliers(data, num_var))

# Eliminar filas con valores atípicos
print(outliers)
print(f"Total de filas con valores atípicos: {sum([len(outlier) for outlier in outliers])}")
print("No se eliminarán los valores atípicos en este ejemplo.")

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')
[       age      job   marital  education default  balance housing loan  \
29158   83  retired   married    primary      no      425      no   no   
29261   75  retired  divorced    primary      no       46      no   no   
29263   75  retired   married    primary      no     3324      no   no   
29322   83  retired   married   tertiary      no     6236      no   no   
29865   75  retired  divorced    primary      no     3881     yes   no   
...    ...      ...       ...        ...     ...      ...     ...  ...   
45163   71  retired   married  secondary      no     2064      no   no   
45191   75  retired  divorced   tertiary      no     3810     yes   no   
45204   73  retired   married  secondary      no     2850      no   no   
45207   71  retired  divorced    primary      no     1729      no   no   
45208   72  retired   married  secondary      no     5715      no   no   

         contact

## 4. Codificación de Variables Categóricas

### 4.1 Reemplazar los valores dicotómicos

In [5]:
# Seleccionar las variables categoricas
cat_vars = data.select_dtypes(include=['object']).columns
print(cat_vars)

# Identificar las variables categóricas binarias
bool_vars = [cat_var for cat_var in cat_vars if data[cat_var].nunique() == 2]

# Convertir las variables categóricas binarias a variables numéricas
dict_map = {'yes': 1, 'no': 0}

for bool_var in bool_vars:
    data_clean[bool_var] = data_clean[bool_var].replace(dict_map)

# Mostrar las primeras filas del conjunto de datos transformado
data_clean.head()

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'y'],
      dtype='object')


C:\Users\HernanCham\AppData\Local\Temp\ipykernel_17528\251865196.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_clean[bool_var] = data_clean[bool_var].replace(dict_map)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0


### 4.2 Codificar el resto de Variables Categoricas

In [6]:
# Convertir variables categóricas a variables dummy
data_clean = pd.get_dummies(data_clean, drop_first=True)

# Mostrar las primeras filas del conjunto de datos transformado
data_clean.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
0,58,0,2143,1,0,5,261,1,-1,0,...,False,False,False,True,False,False,False,False,False,True
1,44,0,29,1,0,5,151,1,-1,0,...,False,False,False,True,False,False,False,False,False,True
2,33,0,2,1,1,5,76,1,-1,0,...,False,False,False,True,False,False,False,False,False,True
3,47,0,1506,1,0,5,92,1,-1,0,...,False,False,False,True,False,False,False,False,False,True
4,33,0,1,0,0,5,198,1,-1,0,...,False,False,False,True,False,False,False,False,False,True


## 5. Escalado de Variables Numéricas

In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Seleccionar solo las columnas numéricas
num_vars = data.select_dtypes(include=['int64']).columns

# Identificar las variables escalables
esc_vars = [var for var in num_vars if var not in bool_vars]

# Elegir el método de escalado (MinMax, Standard, Robust, Log)
scaling_method = 'minmax'  # Cambia esto a 'minmax', 'standard', 'robust', o 'log' según sea necesario

if scaling_method == 'minmax':
    scaler = MinMaxScaler()
    data_clean[esc_vars] = scaler.fit_transform(data_clean[esc_vars])
elif scaling_method == 'standard':
    scaler = StandardScaler()
    data_clean[esc_vars] = scaler.fit_transform(data_clean[esc_vars])
elif scaling_method == 'robust':
    scaler = RobustScaler()
    data_clean[esc_vars] = scaler.fit_transform(data_clean[esc_vars])
elif scaling_method == 'log':
    data_clean[esc_vars] = data_clean[esc_vars].apply(lambda x: np.log(x + 1))

# Mostrar las primeras filas del conjunto de datos escalado
data_clean.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
0,0.519481,0,0.092259,1,0,0.133333,0.053070,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,True
1,0.337662,0,0.073067,1,0,0.133333,0.030704,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,True
2,0.194805,0,0.072822,1,1,0.133333,0.015453,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,True
3,0.376623,0,0.086476,1,0,0.133333,0.018707,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,True
4,0.194805,0,0.072812,0,0,0.133333,0.040260,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,True


## 6. División del Conjunto de Datos

In [8]:
from sklearn.model_selection import train_test_split

# Definir las características (X) y la variable objetivo (y)
X = data_clean.drop('y', axis=1)
y = data_clean['y']

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=8)

# Mostrar las dimensiones de los conjuntos de datos
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (33908, 42)
X_test shape: (11303, 42)
y_train shape: (33908,)
y_test shape: (11303,)


## 7. Guardar Datos Preprocesados

In [9]:
dir_path = '../data/interim'

# Guardar la data_clean
data_clean.to_csv(f'{dir_path}/bank-full.csv', index=False)

# Guardar los conjuntos de datos preprocesados
X_train.to_csv(f'{dir_path}/X_train.csv', index=False)
X_test.to_csv(f'{dir_path}/X_test.csv', index=False)
y_train.to_csv(f'{dir_path}/y_train.csv', index=False)
y_test.to_csv(f'{dir_path}/y_test.csv', index=False)

## 8. Resumen de Preprocesamiento

En esta sección, resume los pasos tomados durante el preprocesamiento de datos, incluyendo:

1. **Imputación de Valores Faltantes**: Se imputaron valores faltantes utilizando la moda para variables categóricas y la mediana para variables numéricas.
2. **Manejo de Valores Atípicos**: Se eliminaron valores atípicos basados en el rango intercuartílico (IQR).
3. **Codificación de Variables Categóricas**: Las variables categóricas fueron convertidas a variables dummy.
4. **Escalado de Variables Numéricas**: Las variables numéricas fueron escaladas usando `MinMaxScaler`.
5. **División de Datos**: Los datos fueron divididos en conjuntos de entrenamiento y prueba con una proporción del 75%-25%.
